<a href="https://colab.research.google.com/github/hetanshipatel/real-time-stock-price-prediction/blob/main/notebooks/01_data_collection_and_eda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install yfinance pandas matplotlib

In [1]:
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime

In [2]:
STOCK_TICKER = input("Enter stock ticker (e.g., AAPL, TSLA): ").upper()
START_DATE = "2015-01-01"

Enter stock ticker (e.g., AAPL, TSLA): NFLX


In [4]:
stock_df = yf.Ticker(STOCK_TICKER).history(start=START_DATE)

if stock_df.empty:
    raise ValueError("Invalid ticker or no data found")

stock_df.reset_index(inplace=True)
stock_df.head()


,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
2769,2026-01-07 00:00:00-05:00,91.559998,92.419998,90.059998,90.730003,36525700,0.0,0.0
2770,2026-01-08 00:00:00-05:00,90.449997,91.250000,89.580002,90.529999,40068700,0.0,0.0
2771,2026-01-09 00:00:00-05:00,90.029999,90.050003,88.320000,89.459999,55579500,0.0,0.0
2772,2026-01-12 00:00:00-05:00,89.690002,90.339996,89.330002,89.410004,36290600,0.0,0.0
2773,2026-01-13 00:00:00-05:00,89.379997,91.150002,89.070000,90.320000,44514058,0.0,0.0


In [5]:
external_tickers = {
    "SP500": "^GSPC",
    "NASDAQ": "^IXIC",
    "VIX": "^VIX",
    "US10Y": "^TNX"
}

external_dfs = []

for name, ticker in external_tickers.items():
    df_ext = yf.Ticker(ticker).history(start=START_DATE)[["Close"]]
    df_ext = df_ext.rename(columns={"Close": name})
    df_ext.reset_index(inplace=True)
    external_dfs.append(df_ext)


In [6]:
df = stock_df.copy()

for ext_df in external_dfs:
    df = pd.merge(df, ext_df, on="Date", how="left")

df.head()


,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,SP500,NASDAQ,VIX,US10Y
0,2015-01-02 00:00:00-05:00,4.915143,5.033143,4.873143,4.984857,134750000,0.0,0.0,2058.199951,4726.810059,17.790001,NaN
1,2015-01-05 00:00:00-05:00,4.925857,4.925857,4.714714,4.731143,181650000,0.0,0.0,2020.579956,4652.569824,19.920000,NaN
2,2015-01-06 00:00:00-05:00,4.734714,4.764000,4.566143,4.650143,160377000,0.0,0.0,2002.609985,4592.740234,21.120001,NaN
3,2015-01-07 00:00:00-05:00,4.734714,4.742143,4.627143,4.674286,98497000,0.0,0.0,2025.900024,4650.470215,19.309999,NaN
4,2015-01-08 00:00:00-05:00,4.712000,4.783571,4.647857,4.778000,96019000,0.0,0.0,2062.139893,4736.189941,17.010000,NaN


In [7]:
today = datetime.today().strftime("%Y-%m-%d")
file_name = f"{STOCK_TICKER}_raw_data_{today}.csv"
df.to_csv(file_name, index=False)
file_name

'NFLX_raw_data_2026-01-13.csv'

In [8]:
from google.colab import files
files.download(file_name)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>